In [139]:
import json
from enum import IntEnum
from src.utils.pb_client import get_pb_client

In [140]:
client = get_pb_client()

class OptionCombinationType(IntEnum):
    """期权交易策略枚举"""
    BULL_CALL_SPREAD = 50        # 认购牛市价差策略
    BEAR_PUT_SPREAD = 51         # 认沽熊市价差策略
    BULL_PUT_SPREAD = 52         # 认沽牛市价差策略
    BEAR_CALL_SPREAD = 53        # 认购熊市价差策略
    SHORT_STRADDLE = 54          # 跨式空头
    SHORT_STRANGLE = 55          # 宽跨式空头
    MARGIN_TO_COVERED = 56       # 保证金开仓转备兑开仓
    COVERED_TO_MARGIN = 57       # 备兑开仓转保证金开仓

def trade(command, symbol, volume):
    data =  {
        "opType": command,  # 50：买入开仓 51：卖出平仓 52：卖出开仓  53：买入平仓
        "orderType": 1101,     
        "orderCode": symbol,        
        "prType": 14,         
        "price": -1,   
        "volume": volume, 
        "strategyName": "打狗棒法",
        "quickTrade": 1,
        "userOrderId": "7416w114037s7c8",
        "user": "v4w3357rsqml48g", #"1ma95x2xvh7qqdv", #
        "accountId": "840092285"
    }
    client.collection("tradeCommands").create(data)

def sell_open(symbol, volume):
    trade(52, symbol, volume)

def buy_open(symbol, volume):
    trade(50, symbol, volume)

def buy_close(symbol, volume):
    trade(53, symbol, volume)

def sell_close(symbol, volume):
    trade(51, symbol, volume)

def cancel(task_id):
    data = {
        "orderType": -100,     
        "orderCode": task_id,        
        "strategyName": "打狗棒法",
        "userOrderId": "7416w114037s7c8",
        "user": "v4w3357rsqml48g", #"1ma95x2xvh7qqdv", #
        "accountId": "840092285"
    }
    client.collection("tradeCommands").create(data)

def make_combination(comd_type,code_1, is_buyer_1, code_2, is_buyer_2, volume):    
    json_obj = {
        f"{code_1}": 48 if is_buyer_1 else 49,
        f"{code_2}": 48 if is_buyer_2 else 49
    }
    # 准备要插入的数据
    data = {
        "opType": comd_type.value, 
        "orderType": -200,     
        "orderCode": json.dumps(json_obj),   
        "volume": volume,     
        "strategyName": "打狗棒法",
        "userOrderId": "7416w114037s7c8",
        "user": "v4w3357rsqml48g", #"1ma95x2xvh7qqdv", #
        "accountId": "840092285"
    }

    client.collection("tradeCommands").create(data)

def release_combination(comb_id):
    data = {
        "orderType": -300,     
        "orderCode": comb_id,        
        "strategyName": "打狗棒法",
        "userOrderId": "7416w114037s7c8",
        "user": "v4w3357rsqml48g", #"1ma95x2xvh7qqdv", #
        "accountId": "840092285"
    }
    client.collection("tradeCommands").create(data)
    

### 下单示例

In [23]:
call_1 = '10009326.SHO' # 6月2700
call_2 = '10009327.SHO' # 6月2750
put_1 = '10009335.SHO' # 6月2700
put_2 = '10009336.SHO' # 6月2750


sell_open(call_1, 1)
sell_open(call_2, 1)
sell_open(put_1, 1)
sell_open(put_2, 1)






### 初始化数据库

In [8]:
def delete_table(client, table_name):
    records = client.collection(table_name).get_full_list()
    for record in records:
        client.collection(table_name).delete(record.id)

delete_table(client, "tradeCommands")
delete_table(client, "positions")
# delete_table(client, "strategyAccount")
delete_table(client, "strategyPositions")
delete_table(client, "strategyCombinations")

### 通过添加deal记录实现下单

In [153]:
def create_deal(client, code, volume, price, direction, offset, remark=''):
    data =  {
        "exchangeId": "SHO",
        "exchangeName": "上证股票期权",
        "instrumentId": code,
        "direction": 48 if direction == "buy" else 49,
        "offsetFlag": 48 if offset == "open" else 49,
        "price": price,
        "optName": ("买入" if direction == "buy" else "卖出") + ('开仓' if offset == 'open' else '平仓'),
        "entrustType": 48,
        "accountId": "840092285",
        "remark": "7416w114037s7c8" if remark == '' else "7416w114037s7c8."+remark,
        "user": "v4w3357rsqml48g",
        "volume": volume,
        }


    client.collection("deals").create(data)

In [154]:
create_deal(client, '10008555', 6, 0.0378, 'sell', 'open')
create_deal(client, '10008555', 5, 0.0378, 'sell', 'open')
create_deal(client, '10008555', 5, 0.0378, 'sell', 'open')
create_deal(client, '10008555', 4, 0.0378, 'buy', 'close')
create_deal(client, '10008555', 9, 0.0378, 'buy', 'close')
create_deal(client, '10008555', 3, 0.0378, 'buy', 'close')

In [155]:

create_deal(client, '10008554', 6, 0.0186, 'buy', 'open')
create_deal(client, '10008554', 5, 0.0186, 'buy', 'open')
create_deal(client, '10008554', 5, 0.0186, 'buy', 'open')
create_deal(client, '10008554', 4, 0.0186, 'sell', 'close')
create_deal(client, '10008554', 9, 0.0186, 'sell', 'close')
create_deal(client, '10008554', 3, 0.0186, 'sell', 'close')